<a href="https://colab.research.google.com/github/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/ResNet18_CIFAR10_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

torch.cuda.is_available()

True

In [2]:
import torch
import random
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

#### ResNet18 모델 정의 및 인스턴스 초기화

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os


# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)  # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)  # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential()  # identity인 경우
        if stride != 1:  # stride가 1이 아니라면, Identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.leaky_relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)  # (핵심) skip connection
        out = F.leaky_relu(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes  # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.leaky_relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# ResNet18 함수 정의
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

#### 데이터셋(Dataset) 다운로드 및 불러오기

In [4]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import ConcatDataset

data_transformations = {
    "basic": transforms.Compose([
        transforms.ToTensor()
    ]),
    "crop_flip": transforms.Compose([  # 크롭 + 뒤집기
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ])
}

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=data_transformations["basic"])
train_aug_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=data_transformations["crop_flip"])
augmented_train_data = ConcatDataset((train_dataset, train_aug_dataset))

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=data_transformations["basic"])

train_loader = torch.utils.data.DataLoader(augmented_train_data, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


#### 환경 설정 및 학습(Training) 함수 정의

In [5]:
device = 'cuda'

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.1
file_name = 'resnet18_cifar10.pt'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)


def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)


def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')


def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 50:
        lr = 0.01
    if epoch >= 75:
        lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

#### 학습(Training) 진행

* 대략 20번의 epoch 이후에도 85%가량의 test accuracy를 얻을 수 있습니다.

In [6]:
# for epoch in range(0, 200):
for epoch in range(0, 100):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.1640625
Current benign train loss: 2.3052728176116943

Current batch: 100
Current benign train accuracy: 0.328125
Current benign train loss: 1.8430147171020508

Current batch: 200
Current benign train accuracy: 0.3671875
Current benign train loss: 1.734334945678711

Current batch: 300
Current benign train accuracy: 0.421875
Current benign train loss: 1.520750880241394

Current batch: 400
Current benign train accuracy: 0.484375
Current benign train loss: 1.4827640056610107

Current batch: 500
Current benign train accuracy: 0.484375
Current benign train loss: 1.4461308717727661

Current batch: 600
Current benign train accuracy: 0.578125
Current benign train loss: 1.2528612613677979

Current batch: 700
Current benign train accuracy: 0.453125
Current benign train loss: 1.419650912284851

Total benign train accuarcy: 42.591
Total benign train loss: 1237.757822394371

[ Test epoch: 0 ]

Test accuarcy: 26.31
Test average l


Test accuarcy: 84.64
Test average loss: 0.004795684675872326
Model Saved!

[ Train epoch: 8 ]

Current batch: 0
Current benign train accuracy: 0.9453125
Current benign train loss: 0.23508912324905396

Current batch: 100
Current benign train accuracy: 0.8984375
Current benign train loss: 0.30829739570617676

Current batch: 200
Current benign train accuracy: 0.8828125
Current benign train loss: 0.3444649279117584

Current batch: 300
Current benign train accuracy: 0.890625
Current benign train loss: 0.29286617040634155

Current batch: 400
Current benign train accuracy: 0.8671875
Current benign train loss: 0.3701498806476593

Current batch: 500
Current benign train accuracy: 0.8828125
Current benign train loss: 0.29838985204696655

Current batch: 600
Current benign train accuracy: 0.8671875
Current benign train loss: 0.2994776666164398

Current batch: 700
Current benign train accuracy: 0.9375
Current benign train loss: 0.17763952910900116

Total benign train accuarcy: 90.76
Total benign t


Total benign train accuarcy: 93.461
Total benign train loss: 149.78798956423998

[ Test epoch: 15 ]

Test accuarcy: 87.03
Test average loss: 0.004245205403864384
Model Saved!

[ Train epoch: 16 ]

Current batch: 0
Current benign train accuracy: 0.96875
Current benign train loss: 0.13746429979801178

Current batch: 100
Current benign train accuracy: 0.921875
Current benign train loss: 0.19844986498355865

Current batch: 200
Current benign train accuracy: 0.96875
Current benign train loss: 0.08254988491535187

Current batch: 300
Current benign train accuracy: 0.9296875
Current benign train loss: 0.2258051037788391

Current batch: 400
Current benign train accuracy: 0.9140625
Current benign train loss: 0.19359035789966583

Current batch: 500
Current benign train accuracy: 0.9375
Current benign train loss: 0.23268958926200867

Current batch: 600
Current benign train accuracy: 0.890625
Current benign train loss: 0.3467247486114502

Current batch: 700
Current benign train accuracy: 0.9296875


Current batch: 700
Current benign train accuracy: 0.9453125
Current benign train loss: 0.13375939428806305

Total benign train accuarcy: 94.31
Total benign train loss: 129.48580473661423

[ Test epoch: 23 ]

Test accuarcy: 87.59
Test average loss: 0.004005274319648743
Model Saved!

[ Train epoch: 24 ]

Current batch: 0
Current benign train accuracy: 0.9765625
Current benign train loss: 0.1255694478750229

Current batch: 100
Current benign train accuracy: 0.96875
Current benign train loss: 0.13759608566761017

Current batch: 200
Current benign train accuracy: 0.921875
Current benign train loss: 0.16736896336078644

Current batch: 300
Current benign train accuracy: 0.96875
Current benign train loss: 0.15993179380893707

Current batch: 400
Current benign train accuracy: 0.921875
Current benign train loss: 0.23886394500732422

Current batch: 500
Current benign train accuracy: 0.9296875
Current benign train loss: 0.23459920287132263

Current batch: 600
Current benign train accuracy: 0.9609


Current batch: 600
Current benign train accuracy: 0.9296875
Current benign train loss: 0.21518313884735107

Current batch: 700
Current benign train accuracy: 0.953125
Current benign train loss: 0.11349624395370483

Total benign train accuarcy: 94.67
Total benign train loss: 122.07595185562968

[ Test epoch: 31 ]

Test accuarcy: 85.73
Test average loss: 0.0050380674213171
Model Saved!

[ Train epoch: 32 ]

Current batch: 0
Current benign train accuracy: 0.9375
Current benign train loss: 0.1927572786808014

Current batch: 100
Current benign train accuracy: 0.9453125
Current benign train loss: 0.15206001698970795

Current batch: 200
Current benign train accuracy: 0.9375
Current benign train loss: 0.17160417139530182

Current batch: 300
Current benign train accuracy: 0.9375
Current benign train loss: 0.11484852433204651

Current batch: 400
Current benign train accuracy: 0.96875
Current benign train loss: 0.131829172372818

Current batch: 500
Current benign train accuracy: 0.9609375
Curren

...


Current batch: 200
Current benign train accuracy: 0.9921875
Current benign train loss: 0.011166872456669807

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.007278140168637037

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.004669281654059887

Current batch: 500
Current benign train accuracy: 1.0
Current benign train loss: 0.0018934198888018727

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.0011999988928437233

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.0025118666235357523

Total benign train accuarcy: 99.857
Total benign train loss: 5.220090632792562

[ Test epoch: 63 ]

Test accuarcy: 94.41
Test average loss: 0.0021534759927541018
Model Saved!

[ Train epoch: 64 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.005466407164931297

Current batch: 100
Current benign train accuracy: 0.984375
Current benign 


Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.0010530294384807348

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.002242207992821932

Current batch: 400
Current benign train accuracy: 0.9921875
Current benign train loss: 0.010378132574260235

Current batch: 500
Current benign train accuracy: 0.9921875
Current benign train loss: 0.0131769310683012

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.003293490968644619

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.002366843866184354

Total benign train accuarcy: 99.872
Total benign train loss: 4.113139361987123

[ Test epoch: 71 ]

Test accuarcy: 94.32
Test average loss: 0.002127048970758915
Model Saved!

[ Train epoch: 72 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.00062795530539006

Current batch: 100
Current benign train accuracy: 1.0
Current benign train


Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.005583625286817551

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.0018983663758262992

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.0017594860401004553

Current batch: 500
Current benign train accuracy: 0.9921875
Current benign train loss: 0.03097200207412243

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.005209070164710283

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.0009052744717337191

Total benign train accuarcy: 99.938
Total benign train loss: 2.226196038303897

[ Test epoch: 79 ]

Test accuarcy: 94.59
Test average loss: 0.0020894056808203458
Model Saved!

[ Train epoch: 80 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.005774447228759527

Current batch: 100
Current benign train accuracy: 1.0
Current benign train 


Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.0004026962851639837

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.0017374006565660238

Current batch: 400
Current benign train accuracy: 0.9921875
Current benign train loss: 0.00952242873609066

Current batch: 500
Current benign train accuracy: 1.0
Current benign train loss: 0.0011974566150456667

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.0025285317096859217

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.0010404582135379314

Total benign train accuarcy: 99.966
Total benign train loss: 1.747300316375913

[ Test epoch: 87 ]

Test accuarcy: 94.65
Test average loss: 0.0020501113757491114
Model Saved!

[ Train epoch: 88 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.003473116783425212

Current batch: 100
Current benign train accuracy: 1.0
Current benign trai


Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.00042965670581907034

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.0008243066840805113

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.0022619764786213636

Current batch: 500
Current benign train accuracy: 1.0
Current benign train loss: 0.0006625605747103691

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.0020382609218358994

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.0009448763448745012

Total benign train accuarcy: 99.975
Total benign train loss: 1.4664882841898361

[ Test epoch: 95 ]

Test accuarcy: 94.79
Test average loss: 0.0020402808867394924
Model Saved!

[ Train epoch: 96 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0010121563682332635

Current batch: 100
Current benign train accuracy: 1.0
Current benign train